In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
ROOT = os.path.join(os.pardir)

DATA_DIR = os.path.join(ROOT, 'data')
DATA_PATH = os.path.join(DATA_DIR, 'data.csv')

sys.path.append(os.path.join(ROOT, "src"))

%load_ext autoreload
%autoreload 2

In [3]:
from dataloaders.preprocess import get_indices_split

# Load Data

In [4]:
data = pd.read_csv(DATA_PATH)
data.head()

,img_path,class,bbox_x1,bbox_y1,bbox_x2,bbox_y2,test
0,car_ims/000001.jpg,AM General Hummer SUV 2000,112,7,853,717,0
1,car_ims/000002.jpg,AM General Hummer SUV 2000,48,24,441,202,0
2,car_ims/000003.jpg,AM General Hummer SUV 2000,7,4,277,180,0
3,car_ims/000004.jpg,AM General Hummer SUV 2000,33,50,197,150,0
4,car_ims/000005.jpg,AM General Hummer SUV 2000,5,8,83,58,0


In [16]:
classes = list(data['class'].unique())

In [6]:
training_data = data[data['test'] == 0]
test_data = data[data['test'] == 1]

# Define Splits

In [7]:
# Use Stratified Sampling
train_indices, val_indices = get_indices_split(training_data, 'class', 0.2)

val_data = training_data.iloc[val_indices]
train_data = training_data.iloc[train_indices]

In [42]:
val_classes = val_data['class'].values

In [43]:
val_labels = [class_to_label[c] for c in val_classes]

In [44]:
val_labels

[116,
 185,
 7,
 183,
 15,
 21,
 121,
 101,
 101,
 13,
 115,
 182,
 39,
 51,
 175,
 153,
 60,
 85,
 73,
 188,
 88,
 107,
 193,
 86,
 151,
 32,
 48,
 27,
 124,
 47,
 111,
 115,
 8,
 77,
 17,
 28,
 124,
 16,
 123,
 54,
 11,
 31,
 54,
 20,
 22,
 171,
 46,
 119,
 192,
 56,
 154,
 177,
 2,
 148,
 39,
 71,
 131,
 20,
 162,
 51,
 120,
 38,
 69,
 118,
 180,
 3,
 116,
 51,
 28,
 108,
 0,
 76,
 143,
 110,
 48,
 8,
 2,
 104,
 81,
 194,
 5,
 181,
 186,
 106,
 48,
 95,
 173,
 69,
 90,
 137,
 99,
 56,
 39,
 61,
 46,
 118,
 21,
 159,
 145,
 78,
 193,
 56,
 72,
 68,
 189,
 73,
 84,
 15,
 54,
 168,
 34,
 107,
 147,
 136,
 92,
 186,
 44,
 108,
 38,
 132,
 191,
 182,
 23,
 99,
 189,
 133,
 27,
 73,
 89,
 104,
 79,
 190,
 80,
 133,
 22,
 25,
 27,
 134,
 19,
 6,
 194,
 165,
 137,
 108,
 1,
 150,
 12,
 169,
 22,
 50,
 140,
 90,
 145,
 56,
 16,
 9,
 27,
 85,
 118,
 66,
 44,
 119,
 43,
 151,
 152,
 31,
 73,
 158,
 43,
 147,
 1,
 3,
 65,
 78,
 79,
 57,
 60,
 88,
 74,
 86,
 8,
 157,
 93,
 193,
 171,
 178,
 106,

# Define Augmentations

In [8]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [9]:
train_img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load Images

In [10]:
train_data.head()

,img_path,class,bbox_x1,bbox_y1,bbox_x2,bbox_y2,test
18,car_ims/000019.jpg,AM General Hummer SUV 2000,86,81,371,252,0
12540,car_ims/012541.jpg,Lamborghini Diablo Coupe 2001,75,260,923,593,0
10880,car_ims/010881.jpg,Hyundai Tucson SUV 2012,111,66,984,800,0
9641,car_ims/009642.jpg,GMC Terrain SUV 2012,32,55,607,403,0
9394,car_ims/009395.jpg,Ford Focus Sedan 2007,11,15,277,157,0


In [11]:
IMG_COL = 'img_path'
LABEL_COL = 'class'
IMG_SIZE = 224
BATCH_SIZE = 32

In [26]:
train_gen = train_img_gen.flow_from_dataframe(
    train_data,
    directory=DATA_DIR,
    x_col=IMG_COL,
    y_col=LABEL_COL,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=classes,
    class_mode='sparse',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

Found 6515 images belonging to 196 classes.


In [27]:
val_gen = val_img_gen.flow_from_dataframe(
    val_data,
    directory=DATA_DIR,
    x_col=IMG_COL,
    y_col=LABEL_COL,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=classes,
    class_mode='sparse',
    batch_size=BATCH_SIZE,
    shuffle=False,
)

Found 1629 images belonging to 196 classes.


In [28]:
test_gen = test_img_gen.flow_from_dataframe(
    test_data,
    directory=DATA_DIR,
    x_col=IMG_COL,
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode=None,
    batch_size=17,
    shuffle=False
)

Found 8041 images.


In [29]:
batch = next(val_gen)

In [30]:
batch[1].shape

(32,)

In [31]:
batch[1]

array([116., 185.,   7., 183.,  15.,  21., 121., 101., 101.,  13., 115.,
       182.,  39.,  51., 175., 153.,  60.,  85.,  73., 188.,  88., 107.,
       193.,  86., 151.,  32.,  48.,  27., 124.,  47., 111., 115.],
      dtype=float32)

In [ ]:
train_gen.n // train_gen.batch_size

In [ ]:
val_gen.n // val_gen.batch_size

# Label Map

In [32]:
class_to_label = train_gen.class_indices

In [34]:
train_gen.class_indices == val_gen.class_indices

True

In [35]:
val_gen.class_indices

{'AM General Hummer SUV 2000': 0,
 'Acura Integra Type R 2001': 1,
 'Acura RL Sedan 2012': 2,
 'Acura TL Sedan 2012': 3,
 'Acura TL Type-S 2008': 4,
 'Acura TSX Sedan 2012': 5,
 'Acura ZDX Hatchback 2012': 6,
 'Aston Martin V8 Vantage Convertible 2012': 7,
 'Aston Martin V8 Vantage Coupe 2012': 8,
 'Aston Martin Virage Convertible 2012': 9,
 'Aston Martin Virage Coupe 2012': 10,
 'Audi 100 Sedan 1994': 11,
 'Audi 100 Wagon 1994': 12,
 'Audi A5 Coupe 2012': 13,
 'Audi R8 Coupe 2012': 14,
 'Audi RS 4 Convertible 2008': 15,
 'Audi S4 Sedan 2007': 16,
 'Audi S4 Sedan 2012': 17,
 'Audi S5 Convertible 2012': 18,
 'Audi S5 Coupe 2012': 19,
 'Audi S6 Sedan 2011': 20,
 'Audi TT Hatchback 2011': 21,
 'Audi TT RS Coupe 2012': 22,
 'Audi TTS Coupe 2012': 23,
 'Audi V8 Sedan 1994': 24,
 'BMW 1 Series Convertible 2012': 25,
 'BMW 1 Series Coupe 2012': 26,
 'BMW 3 Series Sedan 2012': 27,
 'BMW 3 Series Wagon 2012': 28,
 'BMW 6 Series Convertible 2007': 29,
 'BMW ActiveHybrid 5 Sedan 2012': 30,
 'BMW 

In [33]:
class_to_label

{'AM General Hummer SUV 2000': 0,
 'Acura Integra Type R 2001': 1,
 'Acura RL Sedan 2012': 2,
 'Acura TL Sedan 2012': 3,
 'Acura TL Type-S 2008': 4,
 'Acura TSX Sedan 2012': 5,
 'Acura ZDX Hatchback 2012': 6,
 'Aston Martin V8 Vantage Convertible 2012': 7,
 'Aston Martin V8 Vantage Coupe 2012': 8,
 'Aston Martin Virage Convertible 2012': 9,
 'Aston Martin Virage Coupe 2012': 10,
 'Audi 100 Sedan 1994': 11,
 'Audi 100 Wagon 1994': 12,
 'Audi A5 Coupe 2012': 13,
 'Audi R8 Coupe 2012': 14,
 'Audi RS 4 Convertible 2008': 15,
 'Audi S4 Sedan 2007': 16,
 'Audi S4 Sedan 2012': 17,
 'Audi S5 Convertible 2012': 18,
 'Audi S5 Coupe 2012': 19,
 'Audi S6 Sedan 2011': 20,
 'Audi TT Hatchback 2011': 21,
 'Audi TT RS Coupe 2012': 22,
 'Audi TTS Coupe 2012': 23,
 'Audi V8 Sedan 1994': 24,
 'BMW 1 Series Convertible 2012': 25,
 'BMW 1 Series Coupe 2012': 26,
 'BMW 3 Series Sedan 2012': 27,
 'BMW 3 Series Wagon 2012': 28,
 'BMW 6 Series Convertible 2007': 29,
 'BMW ActiveHybrid 5 Sedan 2012': 30,
 'BMW 

In [40]:
label_to_class = dict((v,k) for k,v in class_to_label.items())

In [41]:
label_to_class

{0: 'AM General Hummer SUV 2000',
 1: 'Acura Integra Type R 2001',
 2: 'Acura RL Sedan 2012',
 3: 'Acura TL Sedan 2012',
 4: 'Acura TL Type-S 2008',
 5: 'Acura TSX Sedan 2012',
 6: 'Acura ZDX Hatchback 2012',
 7: 'Aston Martin V8 Vantage Convertible 2012',
 8: 'Aston Martin V8 Vantage Coupe 2012',
 9: 'Aston Martin Virage Convertible 2012',
 10: 'Aston Martin Virage Coupe 2012',
 11: 'Audi 100 Sedan 1994',
 12: 'Audi 100 Wagon 1994',
 13: 'Audi A5 Coupe 2012',
 14: 'Audi R8 Coupe 2012',
 15: 'Audi RS 4 Convertible 2008',
 16: 'Audi S4 Sedan 2007',
 17: 'Audi S4 Sedan 2012',
 18: 'Audi S5 Convertible 2012',
 19: 'Audi S5 Coupe 2012',
 20: 'Audi S6 Sedan 2011',
 21: 'Audi TT Hatchback 2011',
 22: 'Audi TT RS Coupe 2012',
 23: 'Audi TTS Coupe 2012',
 24: 'Audi V8 Sedan 1994',
 25: 'BMW 1 Series Convertible 2012',
 26: 'BMW 1 Series Coupe 2012',
 27: 'BMW 3 Series Sedan 2012',
 28: 'BMW 3 Series Wagon 2012',
 29: 'BMW 6 Series Convertible 2007',
 30: 'BMW ActiveHybrid 5 Sedan 2012',
 31: '

In [54]:
val_gen.classes

[116,
 185,
 7,
 183,
 15,
 21,
 121,
 101,
 101,
 13,
 115,
 182,
 39,
 51,
 175,
 153,
 60,
 85,
 73,
 188,
 88,
 107,
 193,
 86,
 151,
 32,
 48,
 27,
 124,
 47,
 111,
 115,
 8,
 77,
 17,
 28,
 124,
 16,
 123,
 54,
 11,
 31,
 54,
 20,
 22,
 171,
 46,
 119,
 192,
 56,
 154,
 177,
 2,
 148,
 39,
 71,
 131,
 20,
 162,
 51,
 120,
 38,
 69,
 118,
 180,
 3,
 116,
 51,
 28,
 108,
 0,
 76,
 143,
 110,
 48,
 8,
 2,
 104,
 81,
 194,
 5,
 181,
 186,
 106,
 48,
 95,
 173,
 69,
 90,
 137,
 99,
 56,
 39,
 61,
 46,
 118,
 21,
 159,
 145,
 78,
 193,
 56,
 72,
 68,
 189,
 73,
 84,
 15,
 54,
 168,
 34,
 107,
 147,
 136,
 92,
 186,
 44,
 108,
 38,
 132,
 191,
 182,
 23,
 99,
 189,
 133,
 27,
 73,
 89,
 104,
 79,
 190,
 80,
 133,
 22,
 25,
 27,
 134,
 19,
 6,
 194,
 165,
 137,
 108,
 1,
 150,
 12,
 169,
 22,
 50,
 140,
 90,
 145,
 56,
 16,
 9,
 27,
 85,
 118,
 66,
 44,
 119,
 43,
 151,
 152,
 31,
 73,
 158,
 43,
 147,
 1,
 3,
 65,
 78,
 79,
 57,
 60,
 88,
 74,
 86,
 8,
 157,
 93,
 193,
 171,
 178,
 106,